<a href="https://colab.research.google.com/github/sihamlam88/ai-journey/blob/main/Call_Insight_Main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import requests
import os
import matplotlib.pyplot as plt
import io
import base64
from collections import Counter
from PIL import Image

# Hugging Face models
SUMMARY_MODEL = "philschmid/bart-large-cnn-samsum"
SENTIMENT_MODEL = "nlptown/bert-base-multilingual-uncased-sentiment"
HF_TOKEN = ""  # Replace with your own if needed

HEADERS = {"Authorization": f"Bearer {HF_TOKEN}"} if HF_TOKEN else {}

# Hugging Face API call
def query_hf(model, inputs):
    url = f"https://api-inference.huggingface.co/models/{model}"
    payload = {"inputs": inputs}
    try:
        res = requests.post(url, headers=HEADERS, json=payload, timeout=60)
        res.raise_for_status()
        return res.json()
    except Exception as e:
        return {"error": str(e)}

# Convert star-based sentiment to POSITIVE / NEUTRAL / NEGATIVE
def convert_label(label):
    if "1" in label or "2" in label:
        return "NEGATIVE"
    elif "3" in label:
        return "NEUTRAL"
    else:
        return "POSITIVE"

# Generate a pie chart from sentiment counts
def generate_sentiment_chart(sentiments):
    counts = Counter()

    for line in sentiments:
        if "→" in line:
            try:
                label = line.split("→")[1].split("(")[0].strip()
                counts[label] += 1
            except:
                pass

    labels = list(counts.keys())
    values = list(counts.values())

    fig, ax = plt.subplots()
    ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    plt.title("Sentiment Breakdown")

    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)

    return img_str

# Decode base64 image to PIL format
def decode_chart(base64_img):
    decoded = base64.b64decode(base64_img)
    return Image.open(io.BytesIO(decoded))

# Main transcript analysis
def analyze_transcript(text_input):
    # Parse transcript
    turns = []
    for line in text_input.strip().split("\n"):
        if ":" in line:
            speaker, msg = line.split(":", 1)
            turns.append(f"{speaker.strip()}: {msg.strip()}")
    dialogue = "\n".join(turns)

    # Summarization
    summary_res = query_hf(SUMMARY_MODEL, dialogue)
    if isinstance(summary_res, list):
        summary = summary_res[0].get("summary_text", "No summary returned.")
    else:
        return f"Summary failed: {summary_res.get('error', summary_res)}", "", None

    # Sentiment analysis
    sentiments = []
    for turn in turns:
        sent_res = query_hf(SENTIMENT_MODEL, turn)

        try:
            first_item = sent_res[0] if isinstance(sent_res, list) else sent_res
            if isinstance(first_item, list):
                first_item = first_item[0]

            label = first_item.get("label", "")
            score = round(first_item.get("score", 0), 3)
            sentiment = convert_label(label)
            sentiments.append(f"{turn} → {sentiment} ({score})")

        except Exception as e:
            sentiments.append(f"{turn} → Sentiment Error: {str(e)}")

    chart_base64 = generate_sentiment_chart(sentiments)
    return summary, "\n".join(sentiments), decode_chart(chart_base64)

# Gradio UI
demo = gr.Interface(
    fn=analyze_transcript,
    inputs=gr.Textbox(lines=12, label="Paste transcript (Speaker: message)"),
    outputs=[
        gr.Textbox(label="📄 Summary"),
        gr.Textbox(label="📊 Sentiment"),
        gr.Image(type="pil", label="📈 Sentiment Chart")
    ],
    title="📞 Call Analyzer with Hugging Face",
    description="Summarizes and visualizes sentiment in customer support calls"
)

demo.launch(share=True)